# Tarea 2 Jafed Martinez

## Image captioning

In [1]:
import torch
from torch import nn, optim
from torchvision import transforms
from torch.utils.data import Dataset,DataLoader

import os
import json

import numpy as np
from PIL import Image
from random import choice
from nltk.tokenize import TweetTokenizer
import argparse

from IMC.transformer import Transformer
from train import train



Cargando w2v en ingles

In [2]:
def read_w2v(w2v_path):
    w2v = []
    word_idx = {}
    idx_word = []
    with open(w2v_path, 'r') as file:
        for n, line in enumerate(file):
            if n==0:
                vocab_len, w2v_dim = line.split(' ')
            else:
                line = line.split(' ')
                word_idx[line[0]] = n-1
                idx_word.append(line[0])
                w2v.append(list(map(float, line[1:])))
    return w2v, word_idx, idx_word

In [3]:
w2v, word_idx, idx_word = read_w2v('word2vec_lim.txt')

Cargando las anotaciones tokenizadas y con una longitud maxima data

In [4]:
train_ims   = r'C:\Users\jafse\Documents\Maestria cimat\Proyecto tecnologico\YOLOv5-mask\data\coco2017\train2017'
val_ims     = r'C:\Users\jafse\Documents\Maestria cimat\Proyecto tecnologico\YOLOv5-mask\data\coco2017\val2017'
train_caps  = r'C:\Users\jafse\Documents\Maestria cimat\Proyecto tecnologico\YOLOv5-mask\data\coco2017\annotations_2\captions_train2017.json'
val_caps    = r'C:\Users\jafse\Documents\Maestria cimat\Proyecto tecnologico\YOLOv5-mask\data\coco2017\annotations_2\captions_val2017.json'

In [5]:
def load_captions_to_w2v(captions_json, imgs_path, max_n_caption=5, max_cap_len=15, tk=TweetTokenizer(), word_idx=None):
    '''
    captions_json:  Path a las anotaciones
    imgs_path:      Path a las imagenes
    max_n_caption:  Maxima cantidad de captions por imagen
    '''
    ann_dict = {} 
    #Revisando que imagenes estan realmente
    imgs = { img_file for img_file in os.listdir(imgs_path) }
    
    #revisando las captions
    with open(captions_json) as file:
        data = json.load(file)
        for img_json in data["images"]:
            '''
            Algunas imagenes estan en el json pero no en la carpeta por lo tanto 
            es mejor anotar las que si estan y las que no
            '''
            if img_json["file_name"] in imgs: 
                ann_dict[img_json["id"]] = { 
                    "file_name" : img_json["file_name"]
                 } 
                 
        for cap_info in data["annotations"]:
            img_id = cap_info["image_id"]
            if img_id in ann_dict:  #Aqui igual asegurar que la imagen este en los 2 lugares
                #Tokenizar caption
                cap = tk.tokenize(cap_info["caption"])

                #Ajustar len de caption
                if len(cap) < max_cap_len:
                    cap += ['</s>']*(max_cap_len-len(cap))
                elif len(cap) > max_cap_len:
                    cap = cap[:max_cap_len]

                if word_idx is not None:
                    cap = [word_idx['<s>']] + [word_idx[word.lower()] if word.lower() in word_idx else 0 for word in cap  ]
                
                # Cada imagen tiene varios captions
                if "caption" in ann_dict[img_id]:
                    #Agregar caption
                    if len(ann_dict[img_id]["caption"]) <  max_n_caption:
                        ann_dict[img_id]["caption"].append(cap)
                else:
                    ann_dict[img_id]["caption"] =[cap]
                    
    return ann_dict

In [6]:
train_dict = load_captions_to_w2v(train_caps, train_ims, word_idx=word_idx)
val_dict   = load_captions_to_w2v(val_caps, val_ims, word_idx=word_idx)

Creando el dataset

In [7]:
class ICDataset(Dataset):
    def __init__(self, ann_dict, imgs_path, img_size=256):
        '''
        ann_dict:   Diccionario con las anotaciones ya tokenizadas y con embedding
        imgs_path:
        max_len:    Maxima longitud del caption
        words_embedding_idx
        '''
        self.ann_dict   = ann_dict
        self.imgs_path  = imgs_path

        self.img_to_tensor = transforms.functional.pil_to_tensor
        self.resize_img = transforms.Resize((img_size,img_size),antialias=True)

        #Al leer los caption se guarda el id que no esta numerado por lo tanto creo un vector de idx
        self.ann_idx = [x for x in self.ann_dict.keys()]

    def __len__(self):
        return len(self.ann_dict)
    
    def __getitem__(self, i):
        #Cargando imagen
        file_name = self.ann_dict[self.ann_idx[i]]["file_name"]
        img = Image.open(self.imgs_path + "/" + file_name)
        img = img.convert('RGB')#Algunas imagenes estan en blanco y negro
        img = self.resize_img(self.img_to_tensor(img)).float()
        
        #Cargando captions
        capts = self.ann_dict[self.ann_idx[i]]["caption"]
        capts = torch.tensor(capts[0])
        return img, capts


In [8]:
train_dataset = ICDataset(train_dict, train_ims)
val_dataset = ICDataset(val_dict, val_ims)

Creando los dataloaders

In [9]:
BATCH_SIZE = 256

train_dataloader = DataLoader(train_dataset,
                             batch_size=BATCH_SIZE,
                             shuffle=True,
                             drop_last=True)

val_dataloader = DataLoader(val_dataset,
                             batch_size=BATCH_SIZE,
                             shuffle=True,
                             drop_last=True)


Parametros para el entrenamiento

In [10]:
torch.cuda.empty_cache()

In [11]:
lr = 0.0001
epochs = 5
device = torch.device('cuda')
weight_decay = 0.0001
beta1 = 0.9
beta2 = 0.999
chkp_path = 'best'

In [12]:
model = Transformer(vocab_len=len(word_idx), emb_dim=100, d_model=100, d_hidden=1028,
                    n_layers=2, h=2, n_position=50, w2v=torch.tensor(w2v), mask=True)

optimzer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay, betas = (beta1, beta2))

In [13]:
train(model, epochs, train_dataloader, val_dataloader, optimzer, device, chkp_path)


Epoch:  0


Entrenando:  36%|███▌      | 166/462 [03:54<07:35,  1.54s/it]